In [0]:
from __future__ import print_function
import random as rand
import csv
import operator
import gc
import os
from datetime import datetime
from keras.callbacks import EarlyStopping
from keras.models import load_model
import keras.backend as K
from sklearn.metrics import log_loss
import numpy as np

if K.backend() == 'tensorflow':
    import tensorflow as tf
from setuptools import setup, find_packages

from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras import backend as K
from devol import DEvol, GenomeHandler
import pandas as pd
from pyspark.sql.types import *
import inspect
from typing import Callable, List
import pandas as pd
from pyspark.sql import DataFrame, Row, column
from pyspark.sql.functions import lit, pandas_udf, PandasUDFType, array
from pyspark.sql.types import FloatType

In [0]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available: 4

# DEvol On MNIST Results
### Genome Record

In [0]:
# File location and type
file_location = "/dbfs/FileStore/tables/results"
file_type = "parquet"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = pd.read_csv(file_location, sep = delimiter)

display(df)
genomes= df
genomes.index = genomes['Val Accuracy']
genomes = genomes.sort_index().drop(columns=['Val Accuracy', 'Val Loss'])
champion = genomes.tail( n = 1)
best_genome = champion.iloc[0].tolist()

Conv0 active,Conv0 num filters,Conv0 batch normalization,Conv0 activation,Conv0 dropout,Conv0 max pooling,Conv1 active,Conv1 num filters,Conv1 batch normalization,Conv1 activation,Conv1 dropout,Conv1 max pooling,Conv2 active,Conv2 num filters,Conv2 batch normalization,Conv2 activation,Conv2 dropout,Conv2 max pooling,Conv3 active,Conv3 num filters,Conv3 batch normalization,Conv3 activation,Conv3 dropout,Conv3 max pooling,Conv4 active,Conv4 num filters,Conv4 batch normalization,Conv4 activation,Conv4 dropout,Conv4 max pooling,Conv5 active,Conv5 num filters,Conv5 batch normalization,Conv5 activation,Conv5 dropout,Conv5 max pooling,Dense0 active,Dense0 num nodes,Dense0 batch normalization,Dense0 activation,Dense0 dropout,Optimizer,Val Loss,Val Accuracy
1,16,1,1,7,0,0,64,0,0,4,0,0,128,1,1,8,0,1,32,0,0,3,0,0,64,0,0,2,1,0,32,0,0,4,1,0,128,0,0,4,2,0.317605584859848,0.9057999849319458
1,128,1,1,0,0,1,128,0,0,1,0,0,8,0,1,8,0,1,128,1,1,10,2,0,64,1,0,4,1,1,8,1,0,2,2,0,16,1,0,5,0,0.06870068609714508,0.9785000085830688
1,8,1,0,8,0,0,16,0,1,6,2,0,32,1,1,4,0,1,128,1,0,7,1,1,8,1,0,10,2,1,16,0,0,3,1,1,16,0,0,7,2,1.395317554473877,0.8306999802589417
1,128,0,1,5,1,1,8,1,1,7,2,1,16,1,1,2,2,1,256,1,0,1,0,0,128,1,1,6,1,0,32,0,1,6,2,0,128,0,1,3,2,0.415926992893219,0.8693000078201294
1,64,0,0,10,2,0,64,1,0,8,0,1,256,0,0,6,0,0,64,1,0,3,1,0,32,0,1,7,1,1,128,0,1,0,2,0,128,0,1,5,1,0.052957788109779365,0.9829000234603882
1,8,1,1,7,0,0,32,0,1,5,0,1,128,1,1,9,1,1,256,1,0,8,1,0,64,1,0,9,0,0,256,1,0,4,0,0,128,1,1,5,3,1.953291654586792,0.33629998564720154
1,8,0,0,2,0,1,128,1,0,7,0,1,128,0,0,4,0,1,128,0,0,8,2,0,16,1,0,4,2,1,256,0,0,10,1,0,512,1,0,4,2,0.1299610733985901,0.9763000011444092
1,256,1,0,1,1,1,32,1,1,10,2,1,64,0,1,1,1,0,256,1,0,8,2,0,32,1,1,0,2,0,128,0,0,5,2,1,64,1,0,9,3,2.2727112770080566,0.2152000069618225
1,128,1,1,4,0,1,8,1,0,2,0,0,8,1,0,2,0,0,32,0,1,6,1,0,256,1,1,1,2,1,128,1,1,2,0,0,128,0,1,5,1,1.6694078445434568,0.9330000281333924
1,8,0,0,4,1,0,64,1,0,10,2,0,128,0,0,8,1,0,128,0,0,7,0,1,128,0,1,9,0,1,64,1,1,10,0,0,512,0,1,4,0,0.1610460877418518,0.9666000008583068


In [0]:
# File location and type
file_location = "/dbfs/FileStore/tables/results"
file_type = "parquet"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = "\n"

# The applied options are for CSV files. For other file types, these will be ignored.
df = pd.read_csv(file_location, sep = delimiter)
df.columns = ['genes']
display(df)
genomes= df

genes
"1,16,1,1,7,0,0,64,0,0,4,0,0,128,1,1,8,0,1,32,0,0,3,0,0,64,0,0,2,1,0,32,0,0,4,1,0,128,0,0,4,2,0.317605584859848,0.9057999849319458"
"1,128,1,1,0,0,1,128,0,0,1,0,0,8,0,1,8,0,1,128,1,1,10,2,0,64,1,0,4,1,1,8,1,0,2,2,0,16,1,0,5,0,0.06870068609714508,0.9785000085830688"
"1,8,1,0,8,0,0,16,0,1,6,2,0,32,1,1,4,0,1,128,1,0,7,1,1,8,1,0,10,2,1,16,0,0,3,1,1,16,0,0,7,2,1.395317554473877,0.8306999802589417"
"1,128,0,1,5,1,1,8,1,1,7,2,1,16,1,1,2,2,1,256,1,0,1,0,0,128,1,1,6,1,0,32,0,1,6,2,0,128,0,1,3,2,0.415926992893219,0.8693000078201294"
"1,64,0,0,10,2,0,64,1,0,8,0,1,256,0,0,6,0,0,64,1,0,3,1,0,32,0,1,7,1,1,128,0,1,0,2,0,128,0,1,5,1,0.05295778810977936,0.9829000234603882"
"1,8,1,1,7,0,0,32,0,1,5,0,1,128,1,1,9,1,1,256,1,0,8,1,0,64,1,0,9,0,0,256,1,0,4,0,0,128,1,1,5,3,1.953291654586792,0.33629998564720154"
"1,8,0,0,2,0,1,128,1,0,7,0,1,128,0,0,4,0,1,128,0,0,8,2,0,16,1,0,4,2,1,256,0,0,10,1,0,512,1,0,4,2,0.1299610733985901,0.9763000011444092"
"1,256,1,0,1,1,1,32,1,1,10,2,1,64,0,1,1,1,0,256,1,0,8,2,0,32,1,1,0,2,0,128,0,0,5,2,1,64,1,0,9,3,2.2727112770080566,0.2152000069618225"
"1,128,1,1,4,0,1,8,1,0,2,0,0,8,1,0,2,0,0,32,0,1,6,1,0,256,1,1,1,2,1,128,1,1,2,0,0,128,0,1,5,1,1.669407844543457,0.9330000281333923"
"1,8,0,0,4,1,0,64,1,0,10,2,0,128,0,0,8,1,0,128,0,0,7,0,1,128,0,1,9,0,1,64,1,1,10,0,0,512,0,1,4,0,0.1610460877418518,0.9666000008583069"


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
K.set_image_data_format("channels_last")

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
dataset = ((x_train, y_train), (x_test, y_test))

In [0]:
genome_handler = GenomeHandler(max_conv_layers=6, 
                               max_dense_layers=2, # includes final dense layer
                               max_filters=256,
                               max_dense_nodes=1024,
                               input_shape=x_train.shape[1:],
                               n_classes=10)

In [0]:
devol = DEvol(genome_handler)

Genome encoding and metric data stored at Fri Nov 27 07:47:06 2020.csv

In [0]:
import pickle 
pickle_file = open('/dbfs/FileStore/models/simple/pickle','rb')
retrieved_devol = pickle.load(pickle_file)


In [0]:
df.genes = df.genes.astype(str)
#df = df.reset_index()
df = df.tail(n=3)
df

,genes
124,"1,128,1,0,9,1,1,128,0,0,1,0,0,8,0,1,0,2,1,256,..."
125,"1,128,1,1,7,0,1,16,1,0,2,0,0,128,1,1,8,0,1,32,..."
126,"1,128,1,1,4,0,1,16,0,0,4,0,0,128,1,1,8,0,1,32,..."


In [0]:

df = spark.createDataFrame(df, [ "v"])
display(df)

v
"1,128,1,0,9,1,1,128,0,0,1,0,0,8,0,1,0,2,1,256,1,0,5,2,1,8,1,1,7,1,1,8,1,1,0,2,0,64,1,1,2,2,0.8317346572875977,0.958899974822998"
"1,128,1,1,7,0,1,16,1,0,2,0,0,128,1,1,8,0,1,32,1,0,3,0,0,64,0,0,2,1,0,32,0,0,10,2,0,64,1,1,2,2,0.09838128834962845,0.9696999788284302"
"1,128,1,1,4,0,1,16,0,0,4,0,0,128,1,1,8,0,1,32,1,0,1,0,0,128,1,1,6,1,0,32,0,1,0,0,1,1024,0,0,10,2,0.07148057222366333,0.9767000079154968"


In [0]:
#use rdd to parallelize training
def transform_row(row):
  gene = row.v
  gene = gene.split(sep = ',')[:-2]
  result = []
  for item in gene:
    result.append(int(item))
    
  if(genome_handler.is_compatible_genome(result)):
    model, loss, accuracy = retrieved_devol._evaluate(result,1)
  #return accuracy
  return accuracy 
lam = lambda row: transform_row(row)

df.rdd.map(lam).collect()

#genome_handler.is_compatible_genome(best_genome)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4275497423461183> in <module> 
 13 lam = lambda row : transform_row ( row ) 
 14 
 ---> 15 df . rdd . map ( lam ) . collect ( ) 
 16 
 17 #genome_handler.is_compatible_genome(best_genome) 

 /databricks/spark/python/pyspark/rdd.py in collect (self) 
 829 # Default path used in OSS Spark / for non-credential passthrough clusters: 
 830 with SCCallSiteSync ( self . context ) as css : 
 --> 831 sock_info = self . ctx . _jvm . PythonRDD . collectAndServe ( self . _jrdd . rdd ( ) ) 
 832 return list ( _load_from_socket ( sock_info , self . _jrdd_deserializer ) ) 
 833 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 71.0 failed 1 times, most recent failure: Lost task 2.0 in stage 71.0 (TID 466, localhost, executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$3.applyOrElse(PythonRunner.scala:574)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$3.applyOrElse(PythonRunner.scala:563)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:36)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:681)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:659)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:494)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at org.apache.spark.InterruptibleIterator.foreach(InterruptibleIterator.scala:28)
	at scala.collection.generic.Growable$class.$plus$plus$eq(Growable.scala:59)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:104)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:48)
	at scala.collection.TraversableOnce$class.to(TraversableOnce.scala:310)
	at org.apache.spark.InterruptibleIterator.to(InterruptibleIterator.scala:28)
	at scala.collection.TraversableOnce$class.toBuffer(TraversableOnce.scala:302)
	at org.apache.spark.InterruptibleIterator.toBuffer(InterruptibleIterator.scala:28)
	at scala.collection.TraversableOnce$class.toArray(TraversableOnce.scala:289)
	at org.apache.spark.InterruptibleIterator.toArray(InterruptibleIterator.scala:28)
	at org.apache.spark.rdd.RDD$$anonfun$collect$1$$anonfun$15.apply(RDD.scala:997)
	at org.apache.spark.rdd.RDD$$anonfun$collect$1$$anonfun$15.apply(RDD.scala:997)
	at org.apache.spark.SparkContext$$anonfun$runJob$5.apply(SparkContext.scala:2321)
	at org.apache.spark.SparkContext$$anonfun$runJob$5.apply(SparkContext.scala:2321)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$ano

In [0]:

df.rdd.map(lam).collect()

In [0]:

df.rdd.map(lam).collect()

In [0]:

df.rdd.map(lam).collect()

In [0]:

df.rdd.map(lam).collect()

In [0]:

df.rdd.map(lam).collect()

In [0]:

df.rdd.map(lam).collect()

In [0]:

df.rdd.map(lam).collect()

In [0]:

df.rdd.map(lam).collect()

In [0]:

df.rdd.map(lam).collect()

In [0]:

df.rdd.map(lam).collect()

In [0]:
display(df)

In [0]:
dbutils.fs.mkdirs( '/dbfs/FileStore/models/gpu/')
model.save('/dbfs/FileStore/models/gpu/model_gpu_2')

In [0]:
os.listdir('./')

In [0]:
os.rename('./model_gpu_2', './model_gpu_2.model')

In [0]:
os.path.exists('/databricks/driver/model_gpu_2.model')

In [0]:
dbutils.fs.cp( '/databricks/driver/model_gpu_2.model','/dbfs/FileStore/models/gpu/')